In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Lectura de los datasets
df_ratings = pd.read_csv("ratings.csv")

# División entrenamiento/prueba (80/20)
train_data, test_data = train_test_split(df_ratings, test_size=0.2, random_state=42)

# Creamos la matriz de votos solo con datos de entrenamiento
matrix = train_data.pivot(index="userId", columns="movieId", values="rating").fillna(0)

# Calculamos la similaridad entre usuarios
user_similarity = cosine_similarity(matrix)

# Crear mapeo de userIds a índices de matriz
user_id_to_idx = {user_id: idx for idx, user_id in enumerate(matrix.index)}

# Función mejorada con manejo de usuarios/movies desconocidos
def predict_rating(user_id, movie_id, n_neighbors=5):
    try:
        if user_id not in user_id_to_idx:
            return matrix.mean().mean()
            
        user_idx = user_id_to_idx[user_id]
        
        if movie_id not in matrix.columns:
            return matrix.mean().mean()
        
        similar_users = user_similarity[user_idx].argsort()[::-1][1:]
        valid_users = [u for u in similar_users if matrix.iloc[u][movie_id] > 0]
        
        if not valid_users:
            return matrix.loc[:, movie_id].mean() if movie_id in matrix.columns else matrix.mean().mean()
        
        top_users = valid_users[:n_neighbors]
        similarities = user_similarity[user_idx][top_users]
        ratings = matrix.iloc[top_users][movie_id].values
        
        # Manejar suma de similitudes = 0
        similarity_sum = similarities.sum()
        if similarity_sum == 0:
            return matrix.loc[:, movie_id].mean() if movie_id in matrix.columns else matrix.mean().mean()
        
        return np.dot(ratings, similarities) / similarity_sum  # <-- Aquí la corrección
    
    except Exception as e:
        print(f"Error: {str(e)}")
        return matrix.mean().mean()

# Generar predicciones con manejo de errores
predictions = []
true_ratings = []

for _, row in test_data.iterrows():
    try:
        pred = predict_rating(row['userId'], row['movieId'])
        predictions.append(pred)
        true_ratings.append(row['rating'])
    except Exception as e:
        print(f"Fila omitida: {row} - Error: {str(e)}")

# Calcular MAE solo si hay predicciones válidas
if predictions:
    mae = mean_absolute_error(true_ratings, predictions)
    print(f"\nMAE (Error Absoluto Medio): {mae:.4f}")
else:
    print("No se pudo calcular MAE - Sin predicciones válidas")

# Función original de recomendación (modificada para usar datos de entrenamiento)
def recommend_user_based(user_id, n=5):
    user_idx = user_id - 1
    similar_users = user_similarity[user_idx].argsort()[::-1][1:]
    return matrix.iloc[similar_users[:n]].mean().sort_values(ascending=False).head(10)

# Ejemplo de recomendación
user_id = 545
print(f"\nRecomendaciones para el usuario {user_id}:")
recomendaciones = recommend_user_based(user_id, n=5)
print(recomendaciones)


MAE (Error Absoluto Medio): 0.8705

Recomendaciones para el usuario 545:
movieId
6377     2.7
1485     2.3
648      2.0
786      2.0
6539     2.0
733      1.9
344      1.8
33794    1.8
1438     1.8
1393     1.8
dtype: float64
